# Parámetros

In [1]:
year = 2022
week = 8
#path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\Info_POS_clientes\Sell Out - No B2b\Pigmento'
path = r'C:\Users\amartinezgo\genommalabinternacional\Francisco Jose Delfino - Info_POS_clientes\Sell Out - No B2b\Pigmento'

# Libraries

In [2]:
import pandas as pd
from datetime import datetime
import pyodbc
import shutil
import os
from openpyxl import load_workbook

# Weekly Data

## Connections to DWH

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

query_days = '''SELECT TmpFecha
                FROM Gnm_DWH.dbo.Dim_Tiempo
                WHERE TmpAnioSemanaGenomma = {0} AND TmpSemanaAnioGenomma = {1}'''

query_sucs = '''SELECT DISTINCT SUC.SucCodCliente, SUC.SucId
                    FROM (SELECT DISTINCT PaisNombre, CadID FROM Gnm_MasterOp.dbo.vw_EstructuraClientesSegPTVTotal
                    WHERE PaisNombre = 'Argentina') AS CLIE
                LEFT JOIN (
                    SELECT DISTINCT SucId, SucCodCliente, CadId FROM Gnm_MasterOp.dbo.vw_EstructuraSucursalesTotal) AS SUC
                ON CLIE.CadID = SUC.CadID 
                '''

## Load Data

In [4]:
# dates
df_days = pd.read_sql(query_days.format(year, week), conn1)
# id sucursales
df_sucs = pd.read_sql(query_sucs, conn2)

In [5]:
# data Pigmento
file_name = '{0}_{1}_Pigmento.xlsx'
file_name = file_name.format(year, str(week).zfill(2))
df = pd.read_excel(path + '\\' + file_name, header=1)

## Clean Data

In [6]:
df.rename({'EAN':'COD_PROV  ', 'DESCRIPCION':'DESCRIP   '}, axis=1, inplace=True)

In [7]:
# Ignore rows with no data
try:
    not_null = df[df['PROVEEDOR '].isnull()].index.min()
except:
    not_null = df[df['Tot.Stock Val    '].isnull()].index.min()
if pd.isnull(not_null):
    not_null = df.shape[0]
df = df[0:not_null].copy()

# Filter data
df = df[df['COD_PROV  '].notnull()].copy()

df = df[df['ARTICULO  '] != '76387']
df.reset_index(inplace=True, drop=True)

# Change type
df['EAN'] = df['COD_PROV  '].astype('int64')

df.loc[df['ARTICULO  '] == '91251', 'EAN'] = 7798140259794
df.loc[df['ARTICULO  '] == '79158', 'EAN'] = 650240018619

In [8]:
# Columns related to stock units
un_stock_cols = [c for c in df.columns if c.endswith('Stock')]
# Columns related to stock value
val_stock_cols = [c for c in df.columns if c.endswith('Stock Val.')]
# Columns related to sell out units
un_so_cols = [c for c in df.columns if c.endswith('Unid.Vend')]
# Columns related to sell out value
val_so_cols = [c for c in df.columns if c.endswith('Unid.Val. ')]
# Columns related to sku
sku_cols = ['ARTICULO  ', 'EAN', 'DESCRIP   ']

In [9]:
# Remove column 'Total Stock'
if un_stock_cols[-1:][0] == 'Total Stock':
    un_stock_cols.pop(len(un_stock_cols) - 1)

In [10]:
# Unpivote the table into 4 dataframes
# Stock Value
df_sv = df[sku_cols + val_stock_cols].melt(id_vars=sku_cols, value_vars=val_stock_cols, var_name='Suc_vs', value_name='Monto Inv')
# Units Stock
df_us = df[['EAN'] + un_stock_cols].melt(id_vars=['EAN'], value_vars=un_stock_cols, var_name='Suc_uv', value_name='Unidades Inv')
# Sell Out Value
df_sov = df[sku_cols + val_so_cols].melt(id_vars=sku_cols, value_vars=val_so_cols, var_name='Suc_vso', value_name='Monto SO')
# Units Sell Out
df_uso = df[['EAN'] + un_so_cols].melt(id_vars=['EAN'], value_vars=un_so_cols, var_name='Suc_uso', value_name='Unidades SO')

In [11]:
df_sv = df_sv.groupby(sku_cols + ['Suc_vs']).sum().reset_index()
df_us = df_us.groupby(['EAN', 'Suc_uv']).sum().reset_index()
df_sov = df_sov.groupby(sku_cols + ['Suc_vso']).sum().reset_index()
df_uso = df_uso.groupby(['EAN', 'Suc_uso']).sum().reset_index()

In [12]:
# Create IDs
df_sv['ID'] = df_sv['EAN'].astype(str) + df_sv['Suc_vs'].apply(lambda x: x.split(' Stock Val.')[0])
df_us['ID'] = df_us['EAN'].astype(str) + df_us['Suc_uv'].apply(lambda x: x.split(' Stock')[0])
df_sov['ID'] = df_sov['EAN'].astype(str) + df_sov['Suc_vso'].apply(lambda x: x.split(' Unid.Val. ')[0])
df_uso['ID'] = df_uso['EAN'].astype(str) + df_uso['Suc_uso'].apply(lambda x: x.split(' Unid.Vend')[0])
# Merge data stock
data_stock = df_us.merge(df_sv, on='ID', how='outer')
# Merge data sell out
data_so = df_uso.merge(df_sov, on='ID', how='outer')

In [13]:
# Formatting data stock
stock_final = data_stock.copy()
stock_final['Fecha_Stock'] = df_days['TmpFecha'].max()
stock_final['Fecha_Stock'] = stock_final['Fecha_Stock'].dt.date
stock_final['Cadena'] = 'Pigmento'
stock_final['Descripcion_Local'] = stock_final['Suc_uv'].apply(lambda x: x.split(' Stock')[0])
stock_final = stock_final.merge(df_sucs.rename({'SucCodCliente':'Descripcion_Local'}, axis=1), on='Descripcion_Local', how='left')
stock_final.rename({'EAN_x':'EAN o UPC Genomma', 'Unidades Inv':'Unidades', 'ARTICULO  ':'Cod_Prod', 'DESCRIP   ':'Descripcion_Prod', 'Monto Inv':'Valores_Costo', 'SucId':'Cod_Local'}, axis=1, inplace=True)
#stock_final['Cod_Prod'] = stock_final['EAN o UPC Genomma']

In [14]:
stock_final.isnull().sum()

EAN o UPC Genomma    0
Suc_uv               0
Unidades             0
ID                   0
Cod_Prod             0
EAN_y                0
Descripcion_Prod     0
Suc_vs               0
Valores_Costo        0
Fecha_Stock          0
Cadena               0
Descripcion_Local    0
Cod_Local            0
dtype: int64

In [15]:
stock_final = stock_final.pivot_table(index=['Fecha_Stock', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local'], 
                                      values=['Unidades', 'Valores_Costo'], aggfunc='sum').reset_index()

stock_final = stock_final[['Fecha_Stock', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades', 'Valores_Costo']].copy()

In [16]:
stock_final.loc[stock_final['Cod_Local'].isnull(), 'Descripcion_Local'].unique()

array([], dtype=object)

In [17]:
# Formatting data sell out
so_final = data_so.copy()
so_final['Fecha_Venta'] = df_days['TmpFecha'].max()
so_final['Fecha_Venta'] = so_final['Fecha_Venta'].dt.date
so_final['Cadena'] = 'Pigmento'
so_final['Descripcion_Local'] = so_final['Suc_uso'].apply(lambda x: x.split(' Unid.Vend')[0])
so_final = so_final.merge(df_sucs.rename({'SucCodCliente':'Descripcion_Local'}, axis=1), on='Descripcion_Local', how='left')
so_final.rename({'EAN_x':'EAN o UPC Genomma', 'Unidades SO':'Unidades', 'ARTICULO  ':'Cod_Prod de la Cadena', 'DESCRIP   ':'Descripcion_Prod', 'Monto SO':'Valores_Costo', 'SucId':'Cod_Local'}, axis=1, inplace=True)
#so_final['Cod_Prod de la Cadena'] = so_final['EAN o UPC Genomma']

In [18]:
so_final.isnull().sum()

EAN o UPC Genomma        0
Suc_uso                  0
Unidades                 0
ID                       0
Cod_Prod de la Cadena    0
EAN_y                    0
Descripcion_Prod         0
Suc_vso                  0
Valores_Costo            0
Fecha_Venta              0
Cadena                   0
Descripcion_Local        0
Cod_Local                0
dtype: int64

In [19]:
so_final = so_final.pivot_table(index=['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local'],
                                values=['Unidades', 'Valores_Costo'], aggfunc='sum').reset_index()

so_final = so_final[['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades', 'Valores_Costo']].copy()

In [20]:
so_final.loc[so_final['Cod_Local'].isnull(), 'Descripcion_Local'].unique()

array([], dtype=object)

## Export data

In [21]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

Toma el layout de ISV y crea la carpeta correspondiente:

In [22]:
original = '../../../../1Data/2Catalogue/toISV/Layout/Formato No B2B Orig.xlsx'
new_folder = '../../../../1Data/2Catalogue/toISV/{0}/S{1}'.format(year, str(week).zfill(2))

#Create a new directory
if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [23]:
%%time
filename = 'Formato No B2B PIGMENTO Sem {}.xlsx'.format(str(week).zfill(2))
path_export = new_folder + '/' + filename
# Copy layout with name <filename>
shutil.copy(original, path_export)
# Export in different sheets
write_layout(path_export, so_final, True)
write_layout(path_export, stock_final, False)

Wall time: 10.7 s


# Historic Data

year_week = [(2021, 36), (2021, 38), (2021, 39), (2021, 40)]

%%time
#Iterate historic 2020 & 2021
data = {'so':pd.DataFrame(), 'stock':pd.DataFrame()}
locales_alta = []
#year_week = [(y, w) for y in range(2020, 2022) for w in range(1, 53) if int(str(y) + str(w)) < 202131]
year_week = [(2020, w) for w in range(1, 53)]
for year, week in year_week:
    if int(str(year) + str(week)) in [20209, 202013, 202033, 202118]:
        continue
    path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\Info_POS_clientes\Sell Out - No B2b\Pigmento'
    file_name = '{0}_{1}_Pigmento.xlsx'

    df_days = pd.read_sql(query_days.format(year, week), conn1)
    
    df = pd.read_excel(path + '\\' + file_name.format(year, str(week).zfill(2)), header=1)
    
    if not ('ARTICULO  ' in df.columns and 'COD_PROV  ' in df.columns):
        df = pd.read_excel(path + '\\' + file_name.format(year, str(week).zfill(2)))
    
    df.rename({'EAN':'COD_PROV  ', 'DESCRIPCION':'DESCRIP   '}, axis=1, inplace=True)
    
    df.loc[df['ARTICULO  '] == '91251', 'COD_PROV  '] = 7798140259794
    df.loc[df['ARTICULO  '] == '79158', 'COD_PROV  '] = 650240018619
    
    try:
        not_null = df[df['PROVEEDOR '].isnull()].index.min()
    except:
        try:
            not_null = df[df['Tot.Stock Val    '].isnull()].index.min()
        except:
            df = pd.read_excel(path + '\\' + file_name.format(year, str(week).zfill(2)))
            not_null = df[df['Tot.Stock Val    '].isnull()].index.min()
        
    if pd.isnull(not_null):
        not_null = df.shape[0]
    df.rename({'EAN':'COD_PROV  ', 'DESCRIPCION':'DESCRIP   '}, axis=1, inplace=True)
    
    
    df = df[0:not_null].copy()
    df = df[df['COD_PROV  '].notnull()].copy()
    df.reset_index(inplace=True, drop=True)
    df['EAN'] = df['COD_PROV  '].astype('int64')

    un_stock_cols = [c for c in df.columns if c.endswith('Stock')]
    val_stock_cols = [c for c in df.columns if c.endswith('Stock Val.')]
    un_so_cols = [c for c in df.columns if c.endswith('Unid.Vend')]
    val_so_cols = [c for c in df.columns if c.endswith('Unid.Val. ')]
    #'DESCRIPCION'
    sku_cols = ['ARTICULO  ', 'EAN', 'DESCRIP   ']

    if un_stock_cols[-1:][0] == 'Total Stock':
        un_stock_cols.pop(len(un_stock_cols) - 1)

    df_vs = df[sku_cols + val_stock_cols].melt(id_vars=sku_cols, value_vars=val_stock_cols, var_name='Suc_vs', value_name='Monto Inv')
    df_us = df[['EAN'] + un_stock_cols].melt(id_vars=['EAN'], value_vars=un_stock_cols, var_name='Suc_uv', value_name='Unidades Inv')
    df_vso = df[sku_cols + val_so_cols].melt(id_vars=sku_cols, value_vars=val_so_cols, var_name='Suc_vso', value_name='Monto SO')
    df_uso = df[['EAN'] + un_so_cols].melt(id_vars=['EAN'], value_vars=un_so_cols, var_name='Suc_uso', value_name='Unidades SO')
    
    df_vs = df_vs.groupby(sku_cols + ['Suc_vs']).sum().reset_index()
    df_us = df_us.groupby(['EAN', 'Suc_uv']).sum().reset_index()
    df_vso = df_vso.groupby(sku_cols + ['Suc_vso']).sum().reset_index()
    df_uso = df_uso.groupby(['EAN', 'Suc_uso']).sum().reset_index()
    
    df_us['ID'] = df_us['EAN'].astype(str) + df_us['Suc_uv'].apply(lambda x: x.split(' Stock')[0])
    df_vs['ID'] = df_vs['EAN'].astype(str) + df_vs['Suc_vs'].apply(lambda x: x.split(' Stock Val.')[0])
    df_uso['ID'] = df_uso['EAN'].astype(str) + df_uso['Suc_uso'].apply(lambda x: x.split(' Unid.Vend')[0])
    df_vso['ID'] = df_vso['EAN'].astype(str) + df_vso['Suc_vso'].apply(lambda x: x.split(' Unid.Val. ')[0])

    data_stock = df_us.merge(df_vs, on='ID', how='outer')
    data_so = df_uso.merge(df_vso, on='ID', how='outer')
    
    # Formatting data stock
    stock_final = data_stock.copy()
    stock_final['Fecha_Stock'] = df_days['TmpFecha'].max()
    stock_final['Fecha_Stock'] = stock_final['Fecha_Stock'].dt.date
    stock_final['Cadena'] = 'Pigmento'
    stock_final['Descripcion_Local'] = stock_final['Suc_uv'].apply(lambda x: x.split(' Stock')[0])
    stock_final = stock_final.merge(df_sucs.rename({'SucCodCliente':'Descripcion_Local'}, axis=1), on='Descripcion_Local', how='left')
    stock_final.rename({'EAN_x':'EAN o UPC Genomma', 'Unidades Inv':'Unidades', 'ARTICULO  ':'Cod_Prod', 'DESCRIP   ':'Descripcion_Prod', 'Monto Inv':'Valores_Costo', 'SucId':'Cod_Local'}, axis=1, inplace=True)
    stock_final['Cod_Prod'] = stock_final['EAN o UPC Genomma']
    
    stock_final = stock_final.pivot_table(index=['Fecha_Stock', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local'], 
                                      values=['Unidades', 'Valores_Costo'], aggfunc='sum').reset_index()
    stock_final = stock_final[['Fecha_Stock', 'Cadena', 'Cod_Prod', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades', 'Valores_Costo']].copy()

    if len(stock_final.loc[stock_final['Cod_Local'].isnull(), 'Descripcion_Local'].unique()) > 0:
        locales_alta += so_final.loc[so_final['Cod_Local'].isnull(), 'Descripcion_Local'].unique().tolist()
        print('Sucursales a dar de alta en ', year, '-', week)
        print('-'*20, '\n')
    else:
        so_final = data_so.copy()
        so_final['Fecha_Venta'] = df_days['TmpFecha'].max()
        so_final['Fecha_Venta'] = so_final['Fecha_Venta'].dt.date
        so_final['Cadena'] = 'Pigmento'
        so_final['Descripcion_Local'] = so_final['Suc_uso'].apply(lambda x: x.split(' Unid.Vend')[0])
        so_final = so_final.merge(df_sucs.rename({'SucCodCliente':'Descripcion_Local'}, axis=1), on='Descripcion_Local', how='left')
        so_final.rename({'EAN_x':'EAN o UPC Genomma', 'Unidades SO':'Unidades', 'DESCRIP   ':'Descripcion_Prod', 'Monto SO':'Valores_Costo', 'SucId':'Cod_Local'}, axis=1, inplace=True)
        so_final['Cod_Prod de la Cadena'] = so_final['EAN o UPC Genomma']
        
        so_final = so_final.pivot_table(index=['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local'],
                                values=['Unidades', 'Valores_Costo'], aggfunc='sum').reset_index()
        so_final = so_final[['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Descripcion_Local', 'Unidades', 'Valores_Costo']].copy()    
        
        data['stock'] = pd.concat([data['stock'], stock_final])
        data['so'] = pd.concat([data['so'], so_final])
        print('Datos listos en ', year, '-', week)
        print('-'*20, '\n')